## Асимптотический метод

### Асимптотический 90%-ный доверительный интервал среднего

In [1]:
import numpy as np
import pandas as pd

# задаем стартовое значение генератора
# псевдослучайных чисел
np.random.seed(444)
# генеририруем 50 случайных чисел в диапазоне от 45 до 100
income = np.random.uniform(45, 100, size=(50,))
income

array([91.16485761, 90.17122308, 80.32139406, 81.69724401, 47.15269481,
       48.61198136, 60.2075352 , 62.39982769, 55.64758266, 55.84010851,
       91.70623606, 91.32969483, 64.10506002, 85.68582655, 72.13750282,
       71.01564524, 52.20213936, 64.19246693, 95.47880425, 86.51511564,
       50.19910211, 74.43691976, 84.04742879, 75.74817567, 71.11921286,
       97.77624184, 88.16604056, 85.98315068, 98.11275134, 81.00033737,
       86.66306145, 56.79566841, 50.46611201, 48.21183028, 60.23345647,
       97.48078281, 50.2089272 , 84.88498505, 58.85753612, 95.41707634,
       70.70011507, 91.7303194 , 90.3033693 , 96.96710296, 84.39913697,
       74.5518195 , 74.00972834, 66.73663257, 45.54149257, 76.76272365])

In [2]:
# вычисляем среднее значение дохода
mean_income = np.mean(income)
mean_income

74.30188356298876

In [3]:
# записываем информацию о размере выборки
N = 50

In [4]:
# задаем z-значение
z = 1.645
# вычисляем предел погрешности 
err = z * (np.std(income, ddof=1) / np.sqrt(N))

In [5]:
# вычисляем нижнюю границу 90%-ного
# доверительного интервала 
mean_income - err

70.489041592628

In [6]:
# вычисляем верхнюю границу 90%-ного
# доверительного интервала 
mean_income + err

78.11472553334953

In [7]:
# автоматически вычислим 90%-ный асимптотический 
# доверительный интервал
from statsmodels.stats.weightstats import zconfint
zconfint(income, alpha=0.1)

(70.48938086150505, 78.11438626447247)

## Бутстреп-метод

### Учимся вычислять бутстрепированный 95%-ный доверительный интервал среднего

In [8]:
# пишем функцию, генерирующую бутстреп-выборки
def generate_bootstrap(rng, X):
    # получаем индексы наблюдений исходной выборки
    sample_indices = np.arange(X.shape[0])
    # получаем индексы наблюдений бутстреп-выборки, 
    # бутстреп-выборка имеет тот же размер, 
    # что и исходная, отбор с возвращением
    bootstrap_indices = rng.choice(sample_indices,
                                   size=sample_indices.shape[0],
                                   replace=True)
    # формируем бутстреп-выборку
    X_boot = X[bootstrap_indices]
    return X_boot

# создаем контейнер для генератора
# псевдослучайных чисел
rng = np.random.RandomState(42)
    
# создаем пустой список, в который
# будем записывать средние значения
mean_scores_lst = []

# на каждой итерации...
for i in range(1000):
    # формируем бустреп-выборку
    X_boot = generate_bootstrap(rng, income)
    # вычисляем среднее значение дохода
    mean_score = X_boot.mean()
    # кладем вычисленное среднее в список
    mean_scores_lst.append(mean_score)

In [9]:
# пишем функцию, вычисляющую нижнюю и верхнюю границы 
# доверительного интервала для полученного 
# распределения оценок
def compute_conf_interval(stat, alpha):
    boundaries = np.percentile(
        stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [10]:
# вычисляем бутстрепированный 95%-ный 
# доверительный интервал среднего
compute_conf_interval(mean_scores_lst, alpha=0.05)

array([69.48907441, 78.71514224])

## Доверительный интервал метрики качества (на примере AUC-ROC)

### Бутстрепированный 95%-ный доверительный интервал AUC-ROC

In [11]:
# импортируем функцию train_test_split()
from sklearn.model_selection import train_test_split
# импортируем класс DecisionTreeClassifier
# для построения модели дерева
from sklearn.tree import DecisionTreeClassifier
# импортируем функцию roc_auc_score() 
# для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score

In [12]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';') 

# разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

# создаем экземпляр класса DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=123)

# обучаем модель дерева
tree.fit(X_train, y_train)

# вычисляем AUC-ROC для тестовой выборки
auc_test = roc_auc_score(y_test, 
                         tree.predict_proba(X_test)[:, 1])
auc_test

0.9212390292216712

In [13]:
# пишем функцию, вычисляющую стандартную
# ошибку AUC-ROC по методу Хенли-Макнейл
def standard_error_of_auc(auc, y_true):
    """
    Вычисляет стандартную ошибку AUC-ROC по методу Хенли-Макнейл.
    Ссылка https://doi.org/10.1148/radiology.143.1.7063747

    Параметры
    ----------
    auc: float
        Оценка AUC-ROC.

    y_true: одномерный массив формы [n_samples]
        Массив фактических меток.
    """
    neg = y_true.value_counts()[0]
    pos = y_true.value_counts()[1]
    q1 = auc / (2 - auc)
    q2 = (2 * auc ** 2) / (1 + auc)
    return np.sqrt((auc * (1 - auc) + (pos - 1) * (q1 - auc ** 2) + 
                    (neg - 1) * (q2 - auc ** 2)) / (pos * neg))

In [14]:
# вычисляем стандартную ошибку AUC-ROC
se_auc = standard_error_of_auc(auc_test, y_test)
se_auc

0.011946625124979428

In [15]:
# задаем z-значение
z = 1.96
# вычисляем предел погрешности 
err = z * se_auc
err

0.023415385244959678

In [16]:
# получаем асимптотический 95%-ный доверительный 
# интервал AUC-ROC по методу Хенли-Макнейл
np.array([auc_test - err, auc_test + err])

array([0.89782364, 0.94465441])

In [17]:
# создаем массив признаков и массив меток
y_data = data.pop('Response')

In [18]:
# пишем функцию, генерирующую бутстреп-выборки
# и out-of-bag выборки
def generate_bootstrap2(rng, X, y):
    # получаем индексы наблюдений исходной выборки
    sample_indices = np.arange(X.shape[0])
    # получаем индексы наблюдений бутстреп-выборки, 
    # бутстреп-выборка имеет тот же размер, 
    # что и исходная, отбор с возвращением
    bootstrap_indices = rng.choice(sample_indices,
                                   size=sample_indices.shape[0],
                                   replace=True)
    X_boot = X.iloc[bootstrap_indices]
    y_boot = y.iloc[bootstrap_indices]
    X_out_boot = X[~X.index.isin(X_boot.index)]
    y_out_boot = y[~y.index.isin(X_boot.index)]
    # возвращаем выборки
    return X_boot, y_boot, X_out_boot, y_out_boot

In [19]:
# создаем контейнер для генератора
# псевдослучайных чисел
rng = np.random.RandomState(42)
 
# создаем пустой список, в который
# будем записывать значения AUC-ROC
auc_scores_lst = []

# на каждой итерации...
for i in range(1000):
    # формируем бустреп-выборку и out-of-bag выборку
    X_boot, y_boot, X_out_boot, y_out_boot = generate_bootstrap2(
        rng, data, y_data)
    # обучаем модель на бутстреп-выборке
    tree.fit(X_boot, y_boot)
    # записываем значение AUC-ROC
    auc_score = roc_auc_score(y_out_boot, 
                              tree.predict_proba(X_out_boot)[:, 1])
    # кладем значение AUC-ROC в список
    auc_scores_lst.append(auc_score)

In [20]:
# вычисляем бутстрепированный 95%-ный 
# доверительный интервал AUC-ROC
compute_conf_interval(auc_scores_lst, alpha=0.05)

array([0.889707  , 0.93611251])